<a href="https://colab.research.google.com/github/Mudathir-Salahudeen/HEFA-PIPELINE/blob/main/ISS_HEFA_CODE_MAY_2024.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#!/usr/bin/env python
# coding: utf-8

# In[1]:


!pip install azure-storage-blob
!pip install openpyxl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 392.2/392.2 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.4/193.4 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 4.6 MB/s eta 0:00:00


In [2]:
# Azure Libraries
from azure.storage.blob import BlobClient
from azure.storage.blob import BlobServiceClient
from azure.storage.blob import ContainerClient

# Data Maniulation Libraries
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import os
import io
from io import StringIO
import openpyxl





# Set up the Azure Storage Account credentials
connectionString = "DefaultEndpointsProtocol=https;AccountName=kdbstorage123;AccountKey=gwEZVLk9QqhfW/pjLKd7nzziYyDRD8nu8SDGKYkyWpJBk+QeJxHFTrOW97LD+UIMRJYR5mP787Vc8r0CF3XDFQ==;EndpointSuffix=core.windows.net"
# Set up the Blob Service
blob_service_client = BlobServiceClient.from_connection_string(connectionString)
#Set Container name
container_name = 'hefaanalytics'
#Create Container Object
container_client = blob_service_client.get_container_client(container_name)
container_name = 'hefaanalytics'


# Pick the latest file in the input folder (Bolajis latest uploaded file)
raw_data_filepath = 'Output v4-Finalized Orignial/ISS Pipelines/Input/RawData'
container_client = blob_service_client.get_container_client(container_name)
blob_list = container_client.list_blobs(name_starts_with=raw_data_filepath)
# Sort the blob list by last modified time
sorted_blob_list = sorted(blob_list, key=lambda x: x.last_modified, reverse=True)
# Get the latest blob
latest_blob = blob_service_client.get_blob_client(container=container_name, blob=sorted_blob_list[0].name)
# Download the blob content as a string
blob_content = latest_blob.download_blob().content_as_text()
# Create a pandas DataFrame from the string data
df = pd.read_csv(StringIO(blob_content))


# In[4]:


# df = read_file(df) #reads the file into a dataframe using the read_file function.
# factl_namecol = facility_namcol()
# merged_df = group_slice(df, factl_namecol[0])   #merges the teamlead and nonteamlead dataframe back into one after grouping and slicing out the relevant columns for each.
#merged_df.head()

#merged_df.to_csv('bolaji1.csv',index=False)
# In[8]:


# Convert the transformed DataFrame back to a CSV file as bytes
# transformed_csv_bytes = merged_df.to_csv(index=False).encode()
transformed_csv_bytes = df.to_csv(index=False).encode()

output_container_name = 'hefaanalytics/Output v4-Finalized Orignial/ISS Pipelines/Input'
# Get a BlobClient object for the output CSV file
csv_filename = sorted_blob_list[0].name.split("/")[-1]
output_blob_client = blob_service_client.get_blob_client(container=output_container_name, blob=csv_filename)

# Upload the transformed CSV file to the output container
output_blob_client.upload_blob(transformed_csv_bytes, overwrite=True)

print("Transformation and upload complete.")


# In[30]:


def ISS_Transformation():

    PHC_SERVICES = ['id',
    'nutrition_mm13-patient_management_mm13-nutrition_counselling_mm13',
    'pregnancy_mm4-service_provision_mm4-Focused_ANC_mm4',
    'pregnancy_mm4-service_provision_mm4-ganc_mm4',
    'chilhbirth_mm5-serviceprovision_mm5-Prevention_of_Mother_to_Child_Transmission_of_HIV_mm5',
    'Tuberculosis_dot_mm9-service_provision_mm9-Directly_Observed_Treatment_Short_course_mm9',
    'chilhbirth_mm5-serviceprovision_mm5-Care_during_labour_and_childbirth_mm5',
    'adolescence_prepregnancy-serviceprovision_mm3-Family_Planning_mm3',
    'postpartum_mm6-service_provision_mm6-Counselling_and_provision_of_family_planning_services_mm6',
    'pregnancy_mm4-service_provision_mm4-Identification_and_referral_of_pregnant_women_with_complications_mm4',
    'pregnancy_mm4-service_provision_mm4-Referral_of_PW_to_Community_Volunteer',
    'chilhbirth_mm5-serviceprovision_mm5-Detection_and_referral_of_patients_with_complication_mm5',
    'postpartum_mm6-service_provision_mm6-Referral_of_mother_with_complication_mm6',
    'newborn_care_mm7-service_provision_mm7-Referral_mm7',
    'childhealth_mm8-service_provision_mm8-Identification_and_Referral_of_Children_with_danger_signs_mm8',
    'HIV_prevention_and_treatment_mm10-service_provision-referral_service_mm10',
    'malariamanagement_control-service_provision_mm11-referrals_mm11',
    'immunization_mm12-rew_micrplan_in_use_mm12',
    'childhealth_mm8-service_provision_mm8-Integrated_Management_of_Childhood_Illness_mm8',
    'chilhbirth_mm5-serviceprovision_mm5-Prevention_of_Mother_to_Child_Transmission_of_HIV_mm5',
   'immunization_offered'
     ]

    SHF_SERVICES = ['id',
    'ggrp8-hiv_preventn_treatmnt-adolescent_friendly_and_youth_wellness_services-adst-do_clients_receive_hts',
    'ggrp8-hiv_preventn_treatmnt-hiv_service-hivser-is_counselling_and_testing_provided_on_the_same_day_at_the_one_stop_shop',
    'ggrp2-pregnancy-service_provision_b-zzz2-provide_antenatal_care_services',
    'ggrp2-pregnancy-service_provision_b-zzz2-provide_general_antenatal_care_services',
    'ggrp8-hiv_preventn_treatmnt-adolescent_friendly_and_youth_wellness_services-group153-do_clients_receive_sexually_transmitted_infection_screening',
    'ggrp6-childhealth-service_provision_f-group109-q4_provide_comprehensive_care_for_children_infected_with_or_exposed_to_hiv',
    'ggrp8-hiv_preventn_treatmnt-pediatric_art-part-do_hiv_infected_children_with_unsuppressed_viral_load_undergo_eac',
    'ggrp8-hiv_preventn_treatmnt-pediatric_art-part-are_all_confirmed_hiv_infected_children_with_treatment_failure_switched_to_2nd_line_art',
    'ggrp8-hiv_preventn_treatmnt-prevention_of_mother_to_child_transmission-ptt-do_all_pregnant_women_receive_counselling_and_testing_during_antenatal_care',
    'ggrp3-childbirth_labour-service_provision_c-zzz37-provide_care_during_labour_and_delivery',
    'ggrp3-childbirth_labour-service_provision_c-zzz37-manage_obstetric_complications_in_childbirth_and_immediate_post_partum_period',
    'ggrp8-hiv_preventn_treatmnt-treatment_support-group157-family_planning_counseling',
    'ggrp4-postpartum-service_provision_d-group93-provide_referral_services',
    'ggrp5-newborn_care-service_provision_newborn_care_e-group99-offer_referral_services',
    'ggrp2-pregnancy-service_provision_b-zzz2-nutritional_counselling_and_food_demonstration',
    'ggrp10-immunization-service_provision_h-group214-followup_planned_routine_immunization_session_being_conducted',
    'ggrp6-childhealth-service_provision_f-zzz159-manage_common_childhood_illnesses',
    'ggrp6-childhealth-service_provision_f-zzz159-manage_children_with_severe_illness',
    'ggrp6-childhealth-service_provision_f-group110-provide_integrated_management_of_childhood_illness',
    'ggrp4-postpartum-service_provision_d-group93-do_screening_and_initiation_or_continuation_of_arv',
    'ggrp8-hiv_preventn_treatmnt-prevention_of_mother_to_child_transmission-group165-are_all_heis_provided_with_arv_prophylaxis_at_birth',
    'ggrp3-childbirth_labour-service_provision_c-group79-carry_out_initiation_or_continuation_of_hiv_therapy',
    'ggrp3-childbirth_labour-service_provision_c-group79-perform_emergency_caesarean_sections',
    'ggrp4-postpartum-service_provision_d-zzz86-do_counseling_and_provide_family_planning_services_contraceptive_implants',
    'ggrp1-adolescent_prepregnancy-service_provision_a-group62-carry_out_prevention_and_management_of_hiv_sexually_transmitted_infection',
    'ggrp1-adolescent_prepregnancy-service_provision_a-group62-carry_out_appropriate_management_of_infertile_couple_including_hiv_discordant_couples',
    'ggrp6-childhealth-service_provision_f-group109-q4_provide_comprehensive_care_for_children_infected_with_or_exposed_to_hiv',
    'ggrp8-hiv_preventn_treatmnt-pediatric_art-part-do_hiv_infected_children_with_unsuppressed_viral_load_undergo_eac',
    'ggrp8-hiv_preventn_treatmnt-pediatric_art-part-are_all_confirmed_hiv_infected_children_with_treatment_failure_switched_to_2nd_line_art',
    'ggrp8-hiv_preventn_treatmnt-prevention_of_mother_to_child_transmission-ptt-do_all_pregnant_women_receive_counselling_and_testing_during_antenatal_care']


    PHC_EQUIPMENT = ['id',
    'chilhbirth_mm5-equipment_mm5-delivery_bed_mm5',
    'pregnancy_mm4-equipment_mm4-Digital_Blood_Pressure_Device_mm4',
    'Tuberculosis_dot_mm9-service_provision_mm9-microscopes_mm9',
    'chilhbirth_mm5-equipment_mm5-Oxygen_mm5',
    'chilhbirth_mm5-equipment_mm5-Ambu_bag_mask_mm5',
    'pregnancy_mm4-equipment_mm4-Sphygmanometer_mm4',
    'chilhbirth_mm5-equipment_mm5-Sphygmomanometer_and_Stethoscope_mm5',
    'pregnancy_mm4-equipment_mm4-Fetal_and_Adult_Stethoscope_mm4',
    'chilhbirth_mm5-equipment_mm5-Adult_Stethoscope_and_Fetoscope_mm5',
    'chilhbirth_mm5-equipment_mm5-thermometer_mm5',
    'newborn_care_mm7-equipment_mm7-Low_reading_thermomether_mm7',
    'pregnancy_mm4-equipment_mm4-Weighing_Scale_mm4',
    'childhealth_mm8-equipment_mm8-Infant_Weighing_Scales_mm8',
    'nutrition_mm13-equipment_mm13-weighing_scale_mm13',
    'nutrition_mm13-equipment_mm13-weighing_scale_adult_mm13',
    'Tuberculosis_dot_mm9-service_provision_mm9-microscopes_mm9',
    'laboratoryServices-m102-bleachAndDisinfectants',
    'chilhbirth_mm5-equipment_mm5-syringes_mm5',
    'newborn_care_mm7-equipment_mm7-Bulb_syringe_mm7',
    'newborn_care_mm7-equipment_mm7-syringes2ml_mm7',
    'childhealth_mm8-equipment_mm8-Cold_chain_equipments_Fridge_mm8']


    SHF_EQUIPMENT = ['id',
    'ggrp3-childbirth_labour-equipment_and_supplies_c-zzz55-delivery_packs',
    'ggrp5-newborn_care-equipment_and_supplies_e-group105-facility_have_incubators',
    'ggrp2-pregnancy-equipment_and_supplies_b-zzz20-digital_blood_pressure_device_traffic_light_',
    'ggrp3-childbirth_labour-equipment_and_supplies_c-group87-iv_giving_sets',
    'ggrp1-adolescent_prepregnancy-equipment_and_supplies_a-group67-a_theatre_for_surgical_procedure',
    'ggrp1-adolescent_prepregnancy-equipment_and_supplies_a-group68-is_oxygen_available',
    'ggrp3-childbirth_labour-equipment_and_supplies_c-group83-oxygen',
    'ggrp2-pregnancy-equipment_and_supplies_b-group75-sphygmomanometer',
    'ggrp3-childbirth_labour-equipment_and_supplies_c-zzz55-sphygmomanometer',
    'ggrp3-childbirth_labour-equipment_and_supplies_c-group84-adult_stethoscopes_and_fetoscopes',
    'ggrp3-childbirth_labour-equipment_and_supplies_c-group85-clinical_thermometers',
    'ggrp5-newborn_care-equipment_and_supplies_e-zzz133-facility_have_thermometers',
    'ggrp2-pregnancy-equipment_and_supplies_b-zzz20-weighing_scale',
    'ggrp3-childbirth_labour-equipment_and_supplies_c-group85-weighing_scales',
    'ggrp5-newborn_care-equipment_and_supplies_e-group106-facility_have_weighing_scales',
    'ggrp6-childhealth-equipment_and_supplies_f-sss23-facility_have_weighing_scales_and_heightometers',
    'ggrp11-nutrition-equipment_and_supplies_i-group237-weighing_scales__for_infants_and_adults_',
    'ggrp16-laboratoryServices-facility_safety-group260-bleach_and_other_disinfectants',
    'ggrp3-childbirth_labour-equipment_and_supplies_c-group86-gloves_syringes_and_needles']


    # PHC_INFRASTRUCTURE = ['id',
    # 'grp1-ExternalEnvironmentPhysicalInfrastructure-functional_source_of_power_supply_to_this_facility',
    # 'grp1-ExternalEnvironmentPhysicalInfrastructure-health_facility_fenced']

    PHC_INFRASTRUCTURE = ['id','grp1-ExternalEnvironmentPhysicalInfrastructure-Accessible_road_to_the_facility',
    'grp1-ExternalEnvironmentPhysicalInfrastructure-environment_generally_tidy',
    'grp1-ExternalEnvironmentPhysicalInfrastructure-walls_free_of_cracks',
    'grp1-ExternalEnvironmentPhysicalInfrastructure-roof_intact_with_no_leaking',
    'grp1-ExternalEnvironmentPhysicalInfrastructure-facility_well_ventilated',
    'grp1-ExternalEnvironmentPhysicalInfrastructure-potable_water_available',
    'grp1-ExternalEnvironmentPhysicalInfrastructure-sewage_disposal_system_adequate',
    'grp1-ExternalEnvironmentPhysicalInfrastructure-waste_disposal_system_adequate',
    'grp1-ExternalEnvironmentPhysicalInfrastructure-sharp_disposal_system',
    'grp1-ExternalEnvironmentPhysicalInfrastructure-quaterly_rodent_and_pest_control',
    'grp1-ExternalEnvironmentPhysicalInfrastructure-functional_source_of_power_supply_to_this_facility',
    'grp1-ExternalEnvironmentPhysicalInfrastructure-separate_toilet_facilities_for_males_and_females',
    'grp1-ExternalEnvironmentPhysicalInfrastructure-health_facility_fenced',
    'grp1-ExternalEnvironmentPhysicalInfrastructure-facility_space_adequate_for_different_intervention',
    ]


    # SHF_INFRASTRUCTURE = ['id',
    # 'grp1-external_env_and_infrastructure-group8-functional_source_of_power_supply_to_the_facility',
    # 'grp1-external_env_and_infrastructure-group8-facility_fenced']

    SHF_INFRASTRUCTURE = ['id',
    'grp1-external_env_and_infrastructure-ff17-accessible_road_to_the_facility',
       'grp1-external_env_and_infrastructure-ff17-facility_environment_generally_neat_and_tidy',
       'grp1-external_env_and_infrastructure-ff17-walls_free_of_cracks_and_crevices',
       'grp1-external_env_and_infrastructure-group5-roof_intact_with_no_leaking_part',
       'grp1-external_env_and_infrastructure-group5-facility_well_ventilated',
       'grp1-external_env_and_infrastructure-group5-facility_have_potable_water',
       'grp1-external_env_and_infrastructure-group6-facilitys_sewage_disposal_system_adequate',
       'grp1-external_env_and_infrastructure-group6-facilitys_waste_disposal_system_adequate',
       'grp1-external_env_and_infrastructure-group6-facility_have_a_sharp_disposal_system_in_place',
       'grp1-external_env_and_infrastructure-group7-facility_have_a_colour_coded_bin',
       'grp1-external_env_and_infrastructure-group7-facility_carry_out_quarterly_pest_control_activities',
       'grp1-external_env_and_infrastructure-group7-incinerator_in_the_facility',
       'grp1-external_env_and_infrastructure-group8-functional_source_of_power_supply_to_the_facility',
       'grp1-external_env_and_infrastructure-group8-facility_fenced',
       'grp1-external_env_and_infrastructure-group8-neonatal_intensive_care_unit_3_or_special_care_baby_unit_2_',
       'grp1-external_env_and_infrastructure-group9-facility_have_a_functional_and_well_equipped_ambulance',
       'grp1-external_env_and_infrastructure-group9-facilty_have_a_referral_directory',
       'grp1-external_env_and_infrastructure-group10-mobile_phone_network_available_at_the_facilty']


    PHC_STAFF_AVAILABILITY = ['id',
    'grp1-humanResources-staffAvailability-no_registered_midwives_nurses',
    'grp1-humanResources-staffAvailability-no_jchew',
    'grp1-humanResources-staffAvailability-no_labtech',
    'grp1-humanResources-staffAvailability-no_pharmtech',
    'grp1-humanResources-staffAvailability-no_chew',
    'grp1-humanResources-staffAvailability-no_CHO',
    'grp1-humanResources-staffAvailability-no_medical_officers',
    'grp1-humanResources-staffAvailability-num_Community_Volunteers_are_available']


    SHF_STAFF_AVAILABILITY = ['id',
    'grp1-human_resources-staff_availability-group32-followup_how_many_registered_midwives',
    'grp1-human_resources-staff_availability-group33-followup_how_many_registered_general_nurses',
    'grp1-human_resources-staff_availability-group36-followup_how_many_laboratory_technicians',
    'grp1-human_resources-staff_availability-group35-followup_how_many_pharmacy_technicians',
    'grp1-human_resources-staff_availability-group39-followup_how_many_community_health_officers',
    'grp1-human_resources-staff_availability-group31-followup_how_many_medical_officers']


    PHC_STAFF_TRAINING = ['id',
    'grp1-humanResources-staffTraining-num_trained_malaria',
    'grp1-humanResources-staffTraining-num_trained_youthfriendly_centre',
    'grp1-humanResources-staffTraining-num_trained_famplanning',
    'grp1-humanResources-staffTraining-num_trained_HIV',
    'grp1-humanResources-staffTraining-num_trained_imci',
    'grp1-humanResources-staffTraining-num_trained_hmis_tool']


    SHF_STAFF_TRAINING = ['id',
    'grp1-human_resources-staff_training-group57-staff_trained_on_malaria_management',
    'grp1-human_resources-staff_training-group53-staff_trained_on_youth_friendly_health_services',
    'grp1-human_resources-staff_training-group52-staff_trained_on_fp_clms_and_ctu_',
    'grp1-human_resources-staff_training-group50-staff_trained_on_hiv_aids_management',
    'grp1-human_resources-staff_training-group52-staff_trained_on_hts',
    'grp1-human_resources-staff_training-group49-staff_trained_on_integrated_management_of_childhood_illness',
    'grp1-human_resources-staff_training-group54-staff_trained_on_using_nhmis_version_19_tools']


    PHC_LOGISTICS = ['id',
    'logisticsManagement-albendazole',
    'logisticsManagement-mebendazole',
    'logisticsManagement-acts',
    'logisticsManagement-amitriptyline',
    'logisticsManagement-dispersible',
    'logisticsManagement-contraceptive',
    'logisticsManagement-diazepam',
    'logisticsManagement-folic',
    'logisticsManagement-condomMale',
    'logisticsManagement-condomFemale',
    'logisticsManagement-vitaminA',
    'logisticsManagement-tetanus',
    'logisticsManagement-zincORS',
    'logisticsManagement-coldChainItems']


    SHF_LOGISTICS = ['id',
    'ggrp18-logistics_management-data_verification-medicine_supplies_equipment_l-group345-albendazole',
    'ggrp11-nutrition-medicines_and_other_health_technologies_i-group239-albendezole_mebendazole',
    'ggrp18-logistics_management-data_verification-medicine_supplies_equipment_l-group344-act',
    'ggrp18-logistics_management-data_verification-medicine_supplies_equipment_l-group345-dispersible_amoxicillin_tablets',
    'ggrp18-logistics_management-data_verification-medicine_supplies_equipment_l-group348-oral_pills',
    'ggrp18-logistics_management-data_verification-medicine_supplies_equipment_l-group339-iron_and_folic_acid_tablet',
    'ggrp1-adolescent_prepregnancy-equipment_and_supplies_a-ff230-are_male_condoms_available',
    'ggrp1-adolescent_prepregnancy-equipment_and_supplies_a-ff230-are_female_condoms_available',
    'ggrp6-childhealth-medicines_and_other_health_technologies_f-group120-facility_have_vitamin_a',
    'ggrp18-logistics_management-data_verification-medicine_supplies_equipment_l-group341-tetanus_toxoid',
    'ggrp18-logistics_management-data_verification-medicine_supplies_equipment_l-group346-zinc_tablet',
    'ggrp18-logistics_management-data_verification-medicine_supplies_equipment_l-group346-oxytocine']

    #4 create a list for all the PHC intervention column name lists
    ALL_PHC = [PHC_EQUIPMENT,PHC_INFRASTRUCTURE,PHC_LOGISTICS, PHC_SERVICES, PHC_STAFF_AVAILABILITY, PHC_STAFF_TRAINING]

    #5 create a list for all the SHF intervention column name lists
    ALL_SHF = [SHF_EQUIPMENT, SHF_INFRASTRUCTURE, SHF_LOGISTICS, SHF_SERVICES, SHF_STAFF_AVAILABILITY, SHF_STAFF_TRAINING]

    #6 create a list of strings for all intervention names
    PHC_interv = ["PHC EQUIPMENT", "PHC INFRASTRUCTURE", "PHC LOGISTICS", "PHC SERVICES", "PHC STAFF AVAILABILITY", "PHC STAFF TRAINING"]
    SHF_interv = ["SHF EQUIPMENT", "SHF INFRASTRUCTURE", "SHF LOGISTICS", "SHF SERVICES", "SHF STAFF AVAILABILITY", "SHF STAFF TRAINING"]



#     Pick the latest file in the input folder (Bolajis latest uploaded file) from Azure Storage
    raw_data_filepath = 'Output v4-Finalized Orignial/ISS Pipelines/Input'
    container_client = blob_service_client.get_container_client(container_name)
    blob_list = container_client.list_blobs(name_starts_with=raw_data_filepath)
    # Sort the blob list by last modified time
    sorted_blob_list = sorted(blob_list, key=lambda x: x.last_modified, reverse=True)
    # Get the latest blob
    latest_blob = blob_service_client.get_blob_client(container=container_name, blob=sorted_blob_list[0].name)
    # Download the blob content as a string
    blob_content = latest_blob.download_blob().content_as_text()
    # Create a pandas DataFrame from the string data
    df = pd.read_csv(StringIO(blob_content))




    INFRA = df
    # INFRA = pd.read_csv(path) #For local Testing

    #19 rename an immunization column in the services dataset
    if 'immunization_mm12-services_mm12-rew_micrplan_in_use_mm12' in INFRA.columns:
        INFRA.rename({'immunization_mm12-services_mm12-rew_micrplan_in_use_mm12': 'immunization_mm12-rew_micrplan_in_use_mm12'},
                    axis =1, inplace = True)

    #20 renaming multiple columns in the interventions to avoid column mismatch while appending
    # for phc equipment
    equip=['id',
       'chilhbirth_mm5-equipment_mm5-delivery_bed_mm5',
       'pregnancy_mm4-equipment_mm4-Digital_Blood_Pressure_Device_mm4',
       'Tuberculosis_dot_mm9-service_provision_mm9-microscopes_mm9',
       'chilhbirth_mm5-equipment_mm5-Oxygen_mm5',
       'chilhbirth_mm5-equipment_mm5-Ambu_bag_mask_mm5',
       'pregnancy_mm4-equipment_mm4-Sphygmanometer_mm4',
       'chilhbirth_mm5-equipment_mm5-Sphygmomanometer_and_Stethoscope_mm5',
       'pregnancy_mm4-equipment_mm4-Fetal_and_Adult_Stethoscope_mm4',
       'chilhbirth_mm5-equipment_mm5-Adult_Stethoscope_and_Fetoscope_mm5',
       'chilhbirth_mm5-equipment_mm5-thermometer_mm5',
       'newborn_care_mm7-equipment_mm7-Low_reading_thermomether_mm7',
       'pregnancy_mm4-equipment_mm4-Weighing_Scale_mm4',
       'childhealth_mm8-equipment_mm8-Infant_Weighing_Scales_mm8',
       'nutrition_mm13-equipment_mm13-weighing_scale_mm13',
       'nutrition_mm13-equipment_mm13-weighing_scale_adult_mm13',
       'Tuberculosis_dot_mm9-service_provision_mm9-microscopes_mm9.1',
       'laboratoryServices-m102-bleachAndDisinfectants',
       'chilhbirth_mm5-equipment_mm5-syringes_mm5',
       'newborn_care_mm7-equipment_mm7-Bulb_syringe_mm7',
       'newborn_care_mm7-equipment_mm7-syringes2ml_mm7',
        'childhealth_mm8-equipment_mm8-Cold_chain_equipments_Fridge_mm8','year',
       'source']

    #21 for phc services
    serv =['id',
       'nutrition_mm13-patient_management_mm13-nutrition_counselling_mm13',
       'pregnancy_mm4-service_provision_mm4-Focused_ANC_mm4',
       'pregnancy_mm4-service_provision_mm4-ganc_mm4',
       'chilhbirth_mm5-serviceprovision_mm5-Prevention_of_Mother_to_Child_Transmission_of_HIV_mm5',
       'Tuberculosis_dot_mm9-service_provision_mm9-Directly_Observed_Treatment_Short_course_mm9',
       'chilhbirth_mm5-serviceprovision_mm5-Care_during_labour_and_childbirth_mm5',
       'adolescence_prepregnancy-serviceprovision_mm3-Family_Planning_mm3',
       'postpartum_mm6-service_provision_mm6-Counselling_and_provision_of_family_planning_services_mm6',
       'pregnancy_mm4-service_provision_mm4-Identification_and_referral_of_pregnant_women_with_complications_mm4',
       'pregnancy_mm4-service_provision_mm4-Referral_of_PW_to_Community_Volunteer',
       'chilhbirth_mm5-serviceprovision_mm5-Detection_and_referral_of_patients_with_complication_mm5',
       'postpartum_mm6-service_provision_mm6-Referral_of_mother_with_complication_mm6',
       'newborn_care_mm7-service_provision_mm7-Referral_mm7',
       'childhealth_mm8-service_provision_mm8-Identification_and_Referral_of_Children_with_danger_signs_mm8',
       'HIV_prevention_and_treatment_mm10-service_provision-referral_service_mm10',
       'malariamanagement_control-service_provision_mm11-referrals_mm11',
       'immunization_mm12-rew_micrplan_in_use_mm12',
       'childhealth_mm8-service_provision_mm8-Integrated_Management_of_Childhood_Illness_mm8',
       'chilhbirth_mm5-serviceprovision_mm5-Prevention_of_Mother_to_Child_Transmission_of_HIV_mm5.1',
        'immunization_offered',
        'year', 'source']

    #22 for shf services
    shf_serv = ['id',
       'ggrp8-hiv_preventn_treatmnt-adolescent_friendly_and_youth_wellness_services-adst-do_clients_receive_hts',
       'ggrp8-hiv_preventn_treatmnt-hiv_service-hivser-is_counselling_and_testing_provided_on_the_same_day_at_the_one_stop_shop',
       'ggrp2-pregnancy-service_provision_b-zzz2-provide_antenatal_care_services',
       'ggrp2-pregnancy-service_provision_b-zzz2-provide_general_antenatal_care_services',
       'ggrp8-hiv_preventn_treatmnt-adolescent_friendly_and_youth_wellness_services-group153-do_clients_receive_sexually_transmitted_infection_screening',
       'ggrp6-childhealth-service_provision_f-group109-q4_provide_comprehensive_care_for_children_infected_with_or_exposed_to_hiv',
       'ggrp8-hiv_preventn_treatmnt-pediatric_art-part-do_hiv_infected_children_with_unsuppressed_viral_load_undergo_eac',
       'ggrp8-hiv_preventn_treatmnt-pediatric_art-part-are_all_confirmed_hiv_infected_children_with_treatment_failure_switched_to_2nd_line_art',
       'ggrp8-hiv_preventn_treatmnt-prevention_of_mother_to_child_transmission-ptt-do_all_pregnant_women_receive_counselling_and_testing_during_antenatal_care',
       'ggrp3-childbirth_labour-service_provision_c-zzz37-provide_care_during_labour_and_delivery',
       'ggrp3-childbirth_labour-service_provision_c-zzz37-manage_obstetric_complications_in_childbirth_and_immediate_post_partum_period',
       'ggrp8-hiv_preventn_treatmnt-treatment_support-group157-family_planning_counseling',
       'ggrp4-postpartum-service_provision_d-group93-provide_referral_services',
       'ggrp5-newborn_care-service_provision_newborn_care_e-group99-offer_referral_services',
       'ggrp2-pregnancy-service_provision_b-zzz2-nutritional_counselling_and_food_demonstration',
       'ggrp10-immunization-service_provision_h-group214-followup_planned_routine_immunization_session_being_conducted',
       'ggrp6-childhealth-service_provision_f-zzz159-manage_common_childhood_illnesses',
       'ggrp6-childhealth-service_provision_f-zzz159-manage_children_with_severe_illness',
       'ggrp6-childhealth-service_provision_f-group110-provide_integrated_management_of_childhood_illness',
       'ggrp4-postpartum-service_provision_d-group93-do_screening_and_initiation_or_continuation_of_arv',
       'ggrp8-hiv_preventn_treatmnt-prevention_of_mother_to_child_transmission-group165-are_all_heis_provided_with_arv_prophylaxis_at_birth',
       'ggrp3-childbirth_labour-service_provision_c-group79-carry_out_initiation_or_continuation_of_hiv_therapy',
       'ggrp3-childbirth_labour-service_provision_c-group79-perform_emergency_caesarean_sections',
       'ggrp4-postpartum-service_provision_d-zzz86-do_counseling_and_provide_family_planning_services_contraceptive_implants',
       'ggrp1-adolescent_prepregnancy-service_provision_a-group62-carry_out_prevention_and_management_of_hiv_sexually_transmitted_infection',
       'ggrp1-adolescent_prepregnancy-service_provision_a-group62-carry_out_appropriate_management_of_infertile_couple_including_hiv_discordant_couples',
       'ggrp6-childhealth-service_provision_f-group109-q4_provide_comprehensive_care_for_children_infected_with_or_exposed_to_hiv.1',
       'ggrp8-hiv_preventn_treatmnt-pediatric_art-part-do_hiv_infected_children_with_unsuppressed_viral_load_undergo_eac.1',
       'ggrp8-hiv_preventn_treatmnt-pediatric_art-part-are_all_confirmed_hiv_infected_children_with_treatment_failure_switched_to_2nd_line_art.1',
       'ggrp8-hiv_preventn_treatmnt-prevention_of_mother_to_child_transmission-ptt-do_all_pregnant_women_receive_counselling_and_testing_during_antenatal_care.1',
       'year','source']

    # latest Input Filename
    path = sorted_blob_list[0].name
#     path = 'Local Path'   #For local Testing

    #23 operations to be executed if 'file' selected is 1 (1 for PHC)
    if 'PHC' in path:

        #24 Iterate through each intervention for the number intervention times
        for j in range(len(PHC_interv)):

            #25 operations to be done for staff training (index is 5)
            if j == 5:
                #26 create a dataframe of intervention at index 5
                df = pd.DataFrame(INFRA[ALL_PHC[j]])

                #27 recode values greater than 1 to 'Yes' in this particular column
                df['grp1-humanResources-staffTraining-num_trained_imci']= df['grp1-humanResources-staffTraining-num_trained_imci'].apply(lambda x: 'Yes' if x >= 1 else 'No')

                # recode 1 to 'Yes' and 0 to 'No', missing values to 'N_A'
                # Remove the 'id' column temporarily so that the ids will not be affected whn the replacement happens
                df_without_id = df.drop(columns=['id'])
                # Perform the transformation
                df_without_id.replace({1: 'Yes', 0: 'No',  np.nan: 'N/A'}, inplace=True)
                # Add the 'id' column back
                df = pd.concat([df['id'], df_without_id], axis=1)



                #29  extracting the year and source from the file path/source by counting the number of underscores(_) in the path name
                # and assign them to new columns 'year' and 'source'
                # if path.count('_') == 4:
                year = path.split('/')[-1].split('.')[0].split('_')[-1]
                df['year'] = year

                source = path.split('/')[-1].split('.')[0]
                df['source'] = source



                # elif path.count('_') == 5:
                year = path.split('/')[-1].split('.')[0].split('_')[-1]
                df['year'] = year

                source = path.split('/')[-1].split('.')[0]
                df['source'] = source


            if j != 4 and j != 5:#30 operations to be performed for all interventions except 4(PHC Staff availability) and 5(staff training)
                df = pd.DataFrame(INFRA[ALL_PHC[j]]) #31 transform each intervention to dataframe and store as 'df' variable

                # recode 1 to 'Yes' and 0 to 'No', missing values to 'N_A'
                # Remove the 'id' column temporarily so that the ids will not be affected whn the replacement happens
                df_without_id = df.drop(columns=['id'])
                # Perform the transformation
                df_without_id.replace({1: 'Yes', 0: 'No', 204: 'No', np.nan: 'N/A'}, inplace=True)
                # Add the 'id' column back
                df = pd.concat([df['id'], df_without_id], axis=1)

                #32 extract the ISS year from the data source path and assign the year as a new column in the dataset
                # by counting the number of underscore '_' in the path name
                year = path.split('/')[-1].split('.')[0].split('_')[-1]
                df['year'] = year

                source = path.split('/')[-1].split('.')[0]
                df['source'] = source



                # elif path.count('_') == 5:
                year = path.split('/')[-1].split('.')[0].split('_')[-1]
                df['year'] = year

                source = path.split('/')[-1].split('.')[0]
                df['source'] = source


                #34 rename phc equipment column names to already created column names. 0 == phc equipment
                if j == 0:
                    df.columns.values[:]=equip

                #35 rename phc service column names to already created column names. 3 == phc services
                if j == 3:
                    df.columns.values[:]=serv

#           #36 operations to be done for intervention with index 4 (staff availability)
            if j == 4:
                df = pd.DataFrame(INFRA[ALL_PHC[j]])
                df.replace({
                    np.nan:'N/A'},inplace =True)

                year = path.split('/')[-1].split('.')[0].split('_')[-1]
                df['year'] = year

                source = path.split('/')[-1].split('.')[0]
                df['source'] = source



                # elif path.count('_') == 5:
                year = path.split('/')[-1].split('.')[0].split('_')[-1]
                df['year'] = year

                source = path.split('/')[-1].split('.')[0]
                df['source'] = source

            name = f'{PHC_interv[j]}.xlsx'
#             print(name)

            # #38 for new output
            # if name not in os.listdir('/content/sample_data/output/PHC'):
            #     df.to_csv(f'/content/sample_data/output/PHC{PHC_interv[j]}.csv',index=False)
            # #39 to append
            # else:
            #     df2=pd.read_csv(f'/content/sample_data/output/PHC{PHC_interv[j]}.csv')
            #     df2=df2.append(df).to_csv(f'/content/sample_data/output/PHC{PHC_interv[j]}.csv',index=False)

            PHC_out_filepath = 'hefaanalytics/Output v4-Finalized Orignial/ISS Pipelines/Output/PHC'
            blob_client = blob_service_client.get_blob_client(container=PHC_out_filepath, blob=name)



            if not blob_client.exists():
                # If the file doesn't exist, create a new one
                dataframe_Buffer = io.BytesIO()
                df.to_excel(dataframe_Buffer, index=False)
                df_value = dataframe_Buffer.getvalue()
                blob_client.upload_blob(df_value, overwrite=True)
                print("File doesn't exist and has been uploaded")
            else:
                # If the file exists, append the data to the existing file
                stream = blob_client.download_blob().content_as_bytes()
                df2 = pd.read_excel(io.BytesIO(stream))
                df2 = df2.reset_index(drop=True)

                # df2 = df2.append(df)
                # df3 = df2.drop_duplicates(keep='first') # drop duplicates

                # Use pd.concat instead of df.append
                df3 = pd.concat([df2, df]).drop_duplicates(keep='first')

                # Upload excel file
                dataframe_Buffer = io.BytesIO()
                df3.to_excel(dataframe_Buffer, index=False)
                df3_value = dataframe_Buffer.getvalue()
                blob_client.upload_blob(df3_value, overwrite=True)
                print("File exists and has been appended")

        print(path)


    #41 check if the initial input file option  is 2 (for SHF files)
    elif 'SHF' in path:
        for j in range(len(SHF_interv)):
            if j != 4: #42 operations to be performed for all interventions expect staff availability (which is index 4)
                df = pd.DataFrame(INFRA[ALL_SHF[j]])

                # recode 1 to 'Yes' and 0 to 'No', missing values to 'N_A'
                # Remove the 'id' column temporarily so that the ids will not be affected whn the replacement happens
                df_without_id = df.drop(columns=['id'])
                # Perform the transformation
                df_without_id.replace({1: 'Yes', 0: 'No', np.nan: 'N/A'}, inplace=True)
                # Add the 'id' column back
                df = pd.concat([df['id'], df_without_id], axis=1)


                year = path.split('/')[-1].split('.')[0].split('_')[-1]
                df['year'] = year

                source = path.split('/')[-1].split('.')[0]
                df['source'] = source



                # elif path.count('_') == 5:
                year = path.split('/')[-1].split('.')[0].split('_')[-1]
                df['year'] = year

                source = path.split('/')[-1].split('.')[0]
                df['source'] = source



                if j == 3:
                    df.columns.values[:]=shf_serv

            #43 operations to be performed for staff availability intervention (index 4)
            if j == 4:
                df = pd.DataFrame(INFRA[ALL_SHF[j]])

                df.replace({
                    np.nan:'N/A'},inplace =True)

                year = path.split('/')[-1].split('.')[0].split('_')[-1]
                df['year'] = year

                source = path.split('/')[-1].split('.')[0]
                df['source'] = source



                # elif path.count('_') == 5:
                year = path.split('/')[-1].split('.')[0].split('_')[-1]
                df['year'] = year

                source = path.split('/')[-1].split('.')[0]
                df['source'] = source

            name = f'{SHF_interv[j]}.xlsx'


            # #38 for new output
            # if name not in os.listdir('../output/SHF/'):
            #     df.to_csv(f'../output/SHF/{SHF_interv[j]}.csv',index=False)
            # #39 to append
            # else:
            #     df2=pd.read_csv(f'../output/SHF/{SHF_interv[j]}.csv')
            #     df2=df2.append(df).to_csv(f'../output/SHF/{SHF_interv[j]}.csv',index=False)


            SHF_out_filepath = 'hefaanalytics/Output v4-Finalized Orignial/ISS Pipelines/Output/SHF'
            blob_client = blob_service_client.get_blob_client(container=SHF_out_filepath, blob=name)

            # FOR EXCEL FILES
            if not blob_client.exists():
                # If the file doesn't exist, create a new one
                dataframe_Buffer = io.BytesIO()
                df.to_excel(dataframe_Buffer, index=False)
                df_value = dataframe_Buffer.getvalue()
                blob_client.upload_blob(df_value, overwrite=True)
                print("File doesn't exist and has been uploaded")
            else:
                # If the file exists, append the data to the existing file
                stream = blob_client.download_blob().content_as_bytes()
                df2 = pd.read_excel(io.BytesIO(stream))
                # df2 = df2.append(df)
                # df3 = df2.drop_duplicates(keep='first') # drop duplicates

                # Use pd.concat instead of df.append
                df3 = pd.concat([df2, df]).drop_duplicates(keep='first')

                # Upload excel file
                dataframe_Buffer = io.BytesIO()
                df3.to_excel(dataframe_Buffer, index=False)
                df3_value = dataframe_Buffer.getvalue()
                blob_client.upload_blob(df3_value, overwrite=True)
                print("File exists and has been appended")





# In[31]:


ISS_Transformation()


# In[32]:


# Function to upload the files as xlsx
def upload_excel(dataframe, output_filepath, output_excel_filename):
  blob_client = blob_service_client.get_blob_client(container=output_filepath, blob= output_excel_filename)
  dataframe_Buffer = io.BytesIO()
  dataframe.to_excel(dataframe_Buffer, index=False)
  dataframe_value = dataframe_Buffer.getvalue()
  blob_client.upload_blob(dataframe_value, overwrite=True)


# In[33]:


#48 This entire function is designed to append all previous PHC and SHF outputs into 1 single unit file for each intervention
def appending():


    # PHC_fold = sorted(os.listdir('../output/PHC/'))
    # SHF_fold = sorted(os.listdir('../output/SHF/'))

    # List the blobs with .csv extension in the PHC folder
    PHC_fold = sorted([blob.name for blob in blob_service_client.get_container_client(container_name).list_blobs(name_starts_with='Output v4-Finalized Orignial/ISS Pipelines/Output/PHC/') if blob.name.endswith('.xlsx')])
    # List the blobs with .csv extension in the SHF folder
    SHF_fold = sorted([blob.name for blob in blob_service_client.get_container_client(container_name).list_blobs(name_starts_with='Output v4-Finalized Orignial/ISS Pipelines/Output/SHF/') if blob.name.endswith('.xlsx')])


    for i in range(len(PHC_fold)):
        # phc_file = pd.read_csv(f"../output/PHC/{PHC_fold[i]}")
        # shf_file = pd.read_csv(f"../output/SHF/{SHF_fold[i]}")

        # Download the PHC Excel file from Azure Blob Storage
        phc_blob_client = blob_service_client.get_blob_client(container_name, PHC_fold[i])
        stream = phc_blob_client.download_blob().content_as_bytes()
        phc_file = pd.read_excel(io.BytesIO(stream))

        # Download the SHF file from Azure Blob Storage
        shf_blob_client = blob_service_client.get_blob_client(container_name, SHF_fold[i])
        stream = shf_blob_client.download_blob().content_as_bytes()
        shf_file = pd.read_excel(io.BytesIO(stream))


        #52 operations to be performed if the intervention input is 1
        if 'EQUIPMENT' in PHC_fold[i] and 'EQUIPMENT' in SHF_fold[i]:

            # 53 rename the mapped SHF column names to be uniform with the PHC column names
            shf_file.rename({'ggrp3-childbirth_labour-equipment_and_supplies_c-zzz55-delivery_packs':'chilhbirth_mm5-equipment_mm5-delivery_bed_mm5',
                    'ggrp1-adolescent_prepregnancy-equipment_and_supplies_a-group68-is_oxygen_available':'chilhbirth_mm5-equipment_mm5-Oxygen_mm5',
                    'ggrp3-childbirth_labour-equipment_and_supplies_c-group83-oxygen':'chilhbirth_mm5-equipment_mm5-Ambu_bag_mask_mm5',
                    'ggrp2-pregnancy-equipment_and_supplies_b-group75-sphygmomanometer':'pregnancy_mm4-equipment_mm4-Sphygmanometer_mm4',
                    'ggrp3-childbirth_labour-equipment_and_supplies_c-zzz55-sphygmomanometer':'chilhbirth_mm5-equipment_mm5-Sphygmomanometer_and_Stethoscope_mm5',
                    'ggrp3-childbirth_labour-equipment_and_supplies_c-group84-adult_stethoscopes_and_fetoscopes':'pregnancy_mm4-equipment_mm4-Fetal_and_Adult_Stethoscope_mm4',
                    'ggrp3-childbirth_labour-equipment_and_supplies_c-group85-clinical_thermometers':'chilhbirth_mm5-equipment_mm5-thermometer_mm5',
                    'ggrp5-newborn_care-equipment_and_supplies_e-zzz133-facility_have_thermometers':'newborn_care_mm7-equipment_mm7-Low_reading_thermomether_mm7',
                    'ggrp16-laboratoryServices-facility_safety-group260-bleach_and_other_disinfectants':'laboratoryServices-m102-bleachAndDisinfectants',
                    'ggrp2-pregnancy-equipment_and_supplies_b-zzz20-digital_blood_pressure_device_traffic_light_':'pregnancy_mm4-equipment_mm4-Digital_Blood_Pressure_Device_mm4',
                    'ggrp3-childbirth_labour-equipment_and_supplies_c-group85-weighing_scales':'childhealth_mm8-equipment_mm8-Infant_Weighing_Scales_mm8',
                    'ggrp2-pregnancy-equipment_and_supplies_b-zzz20-weighing_scale':'pregnancy_mm4-equipment_mm4-Weighing_Scale_mm4'}, axis=1, inplace = True)


            #54 concatenate / append both PHC and SHF files and output as a single CSV file
            # pd.concat([phc_file, shf_file], axis=0, ignore_index=True).to_csv(f'../output/merged/Complete_Equipment.csv')

            equipment_df = pd.concat([phc_file, shf_file], axis=0, ignore_index=True)
            equipment_df2 = equipment_df.drop_duplicates(keep='first') # remove duplicates

            #reposition year and source
            yr = equipment_df2.pop('year')
            sc = equipment_df2.pop('source')
            id = equipment_df2.pop('id')
            equipment_df2=pd.concat([id,equipment_df2,yr,sc],axis=1)
            # Upload df as xlsx to Azure
            upload_excel(equipment_df2, "hefaanalytics/Output v4-Finalized Orignial/ISS Pipelines/Output/merged", "ISS_Equipment.xlsx")

        # operations to be performed if intervention input by the user is 2
        elif 'INFRASTRUCTURE' in PHC_fold[i] and 'INFRASTRUCTURE' in SHF_fold[i]:
            # rename all mapped SHF column names to be uniform with PHC column names
            shf_file.rename({'grp1-external_env_and_infrastructure-group8-facility_fenced':'grp1-ExternalEnvironmentPhysicalInfrastructure-health_facility_fenced',

            'grp1-external_env_and_infrastructure-ff17-accessible_road_to_the_facility':'grp1-ExternalEnvironmentPhysicalInfrastructure-Accessible_road_to_the_facility',
            'grp1-external_env_and_infrastructure-group6-facilitys_sewage_disposal_system_adequate':'grp1-ExternalEnvironmentPhysicalInfrastructure-sewage_disposal_system_adequate',
            'grp1-external_env_and_infrastructure-group5-facility_have_potable_water':'grp1-ExternalEnvironmentPhysicalInfrastructure-potable_water_available',
            'grp1-external_env_and_infrastructure-group6-facility_have_a_sharp_disposal_system_in_place':'grp1-ExternalEnvironmentPhysicalInfrastructure-sharp_disposal_system',
            'grp1-external_env_and_infrastructure-group5-facility_well_ventilated':'grp1-ExternalEnvironmentPhysicalInfrastructure-facility_well_ventilated',
            'grp1-external_env_and_infrastructure-group5-roof_intact_with_no_leaking_part':'grp1-ExternalEnvironmentPhysicalInfrastructure-roof_intact_with_no_leaking',
            'grp1-external_env_and_infrastructure-ff17-walls_free_of_cracks_and_crevices':'grp1-ExternalEnvironmentPhysicalInfrastructure-walls_free_of_cracks',
            'grp1-external_env_and_infrastructure-group7-facility_carry_out_quarterly_pest_control_activities':'grp1-ExternalEnvironmentPhysicalInfrastructure-quaterly_rodent_and_pest_control',
            'grp1-external_env_and_infrastructure-group8-functional_source_of_power_supply_to_the_facility':'grp1-ExternalEnvironmentPhysicalInfrastructure-functional_source_of_power_supply_to_this_facility',
            'grp1-external_env_and_infrastructure-group6-facilitys_waste_disposal_system_adequate':'grp1-ExternalEnvironmentPhysicalInfrastructure-waste_disposal_system_adequate',
            'grp1-external_env_and_infrastructure-ff17-facility_environment_generally_neat_and_tidy': 'grp1-ExternalEnvironmentPhysicalInfrastructure-environment_generally_tidy'
            }, axis=1, inplace = True)

            # concatenate / append both PHC and SHF files and output as a single CSV file
            # pd.concat([phc_file,shf_file], axis=0, ignore_index=True).to_csv(f'../output/merged/Complete_Infrastructure.csv')

            infra_df = pd.concat([phc_file,shf_file], axis=0, ignore_index=True)
            infra_df2 = infra_df.drop_duplicates(keep='first') # remove duplicates


            #reposition year and source
            yr = infra_df2.pop('year')
            sc = infra_df2.pop('source')
            id = infra_df2.pop('id')
            infra_df2=pd.concat([id,infra_df2,yr,sc],axis=1)

            # Upload df as xlsx to Azure
            upload_excel(infra_df2, "hefaanalytics/Output v4-Finalized Orignial/ISS Pipelines/Output/merged", "infrastructure_pipeline.xlsx")

        # operations to be performed if intervention input by the user is 3
        elif 'LOGISTICS' in PHC_fold[i] and 'LOGISTICS' in SHF_fold[i]:
            # rename all mapped SHF column names for the intervention to be uniform with that of the PHC column names
            shf_file.rename({
            'ggrp18-logistics_management-data_verification-medicine_supplies_equipment_l-group345-albendazole':'logisticsManagement-albendazole',
            'ggrp11-nutrition-medicines_and_other_health_technologies_i-group239-albendezole_mebendazole':'logisticsManagement-mebendazole',
            'ggrp18-logistics_management-data_verification-medicine_supplies_equipment_l-group344-act':'logisticsManagement-acts',
            'ggrp18-logistics_management-data_verification-medicine_supplies_equipment_l-group345-dispersible_amoxicillin_tablets':'logisticsManagement-dispersible',
            'ggrp18-logistics_management-data_verification-medicine_supplies_equipment_l-group348-oral_pills':'logisticsManagement-contraceptive',
            'ggrp18-logistics_management-data_verification-medicine_supplies_equipment_l-group339-iron_and_folic_acid_tablet':'logisticsManagement-folic',
            'ggrp1-adolescent_prepregnancy-equipment_and_supplies_a-ff230-are_male_condoms_available':'logisticsManagement-condomMale',
            'ggrp1-adolescent_prepregnancy-equipment_and_supplies_a-ff230-are_female_condoms_available':'logisticsManagement-condomFemale',
            'ggrp6-childhealth-medicines_and_other_health_technologies_f-group120-facility_have_vitamin_a':'logisticsManagement-vitaminA',
            'ggrp18-logistics_management-data_verification-medicine_supplies_equipment_l-group341-tetanus_toxoid':'logisticsManagement-tetanus',
            'ggrp18-logistics_management-data_verification-medicine_supplies_equipment_l-group346-zinc_tablet':'logisticsManagement-zincORS',
            'ggrp18-logistics_management-data_verification-medicine_supplies_equipment_l-group346-oxytocine':'logisticsManagement-coldChainItems'},axis=1, inplace=True)


            # concatenate / append both PHC and SHF files and output as a single CSV file
            # pd.concat([phc_file,shf_file],axis=0, ignore_index=True).to_csv(f'../output/merged/Complete_Logistics.csv')

            logistics_df = pd.concat([phc_file,shf_file],axis=0, ignore_index=True)
            logistics_df2 = logistics_df.drop_duplicates(keep='first') # remove duplicates


             #reposition year and source
            yr = logistics_df2.pop('year')
            sc = logistics_df2.pop('source')
            id = logistics_df2.pop('id')
            logistics_df2=pd.concat([id,logistics_df2,yr,sc],axis=1)

            # Upload df as xlsx to Azure
            upload_excel(logistics_df2, "hefaanalytics/Output v4-Finalized Orignial/ISS Pipelines/Output/merged", "Logistics_management_pipeline.xlsx")

        # operations to be performed if the intervention input by the user is 4
        elif 'SERVICES' in PHC_fold[i] and 'SERVICES' in SHF_fold[i]:
            # rename all mapped SHF column names to be uniform with the PHC intervention column names
            shf_file.rename({
            'ggrp3-childbirth_labour-service_provision_c-zzz37-provide_care_during_labour_and_delivery':'chilhbirth_mm5-serviceprovision_mm5-Care_during_labour_and_childbirth_mm5',
            'ggrp2-pregnancy-service_provision_b-zzz2-nutritional_counselling_and_food_demonstration':'nutrition_mm13-patient_management_mm13-nutrition_counselling_mm13',
            'ggrp10-immunization-service_provision_h-group214-followup_planned_routine_immunization_session_being_conducted':'immunization_mm12-rew_micrplan_in_use_mm12'
            }, axis =1, inplace = True)

            # concatenate / append both PHC and SHF files and output as a single CSV file
            # pd.concat([phc_file,shf_file],axis=0, ignore_index=True).to_csv(f'../output/merged/Complete_Services.csv')

            services_df = pd.concat([phc_file,shf_file],axis=0, ignore_index=True)
            services_df2 = services_df.drop_duplicates(keep='first') #remove duplicates


            #reposition year and source
            yr = services_df2.pop('year')
            sc = services_df2.pop('source')
            id = services_df2.pop('id')
            services_df2=pd.concat([id,services_df2,yr,sc],axis=1)


                 #rename columns with truncated names
            services_df3=services_df2.rename({"ggrp8-hiv_preventn_treatmnt-adolescent_friendly_and_youth_wellness_services-group153-do_clients_receive_sexually_transmitted_infection_screening":"grp8-hiv_prvntn_trtmnt-adlscnt_frndly_&_youth_welnes_servics-grp153-do_clients_recive_sxualy_trnsmitd_infctn_screning",

            "ggrp8-hiv_preventn_treatmnt-pediatric_art-part-are_all_confirmed_hiv_infected_children_with_treatment_failure_switched_to_2nd_line_art":"grp8-hiv_prvntn_treatmnt-pdiatrc_art-prt-are_al_cnfrmed_hiv_infctd_chldrn_wit_tretmnt_failre_swtchd_to_2nd_line_art",

            "ggrp8-hiv_preventn_treatmnt-prevention_of_mother_to_child_transmission-ptt-do_all_pregnant_women_receive_counselling_and_testing_during_antenatal_care":"grp8-hiv_prevntn_treatmnt-prevntn_of_mthr_to_chld_trnsmssn-ptt-do_al_prgnnt_womn_recive_cnseling_&_tstng_durng_antnatl_cre",

            "ggrp3-childbirth_labour-service_provision_c-zzz37-manage_obstetric_complications_in_childbirth_and_immediate_post_partum_period":"grp3-chldbrth_labr-srvce_prvisn_c-z37-mnage_obstetrc_cmplicatns_in_chldbrth_&_imediate_pst_partm_perod",

            "ggrp8-hiv_preventn_treatmnt-prevention_of_mother_to_child_transmission-group165-are_all_heis_provided_with_arv_prophylaxis_at_birth":"grp8-hiv_prvntn_treatmnt-prvntn_of_mothr_to_chld_trnsmisn-grp165-are_al_his_prvidd_wth_arv_prphylxis_at_brth",

            "ggrp1-adolescent_prepregnancy-service_provision_a-group62-carry_out_prevention_and_management_of_hiv_sexually_transmitted_infection":"grp1-adlscnt_prepregnancy-servce_prvisn_a-grp62-cary_out_prevntn_&_managmnt_of_hiv_sxualy_trnsmitd_infctn",

            "ggrp1-adolescent_prepregnancy-service_provision_a-group62-carry_out_appropriate_management_of_infertile_couple_including_hiv_discordant_couples":"grp1-adolscnt_preprgnncy-service_prvisn_a-grp62-cary_out_apropriate_mnagmnt_of_infrtile_couple_incldng_hiv_dscrdnt_coupls",

            "ggrp8-hiv_preventn_treatmnt-pediatric_art-part-are_all_confirmed_hiv_infected_children_with_treatment_failure_switched_to_2nd_line_art.1":"grp8-hiv_prevntn_treatmnt-pediatrc_art-prt-are_all_cnfirmd_hiv_infctd_childrn_with_treatmnt_failre_swtchd_to_2nd_lne_art1",

            "ggrp8-hiv_preventn_treatmnt-prevention_of_mother_to_child_transmission-ptt-do_all_pregnant_women_receive_counselling_and_testing_during_antenatal_care.1":"grp8-hiv_prevntn_trtmnt-prventn_of_mothr_to_child_trnsmisn-pt-do_al_prgnnt_women_recive_cnseling_&_tstng_drng_antnatl_cre1"}, axis=1)


            # Upload df as xlsx to Azure
            upload_excel(services_df3, "hefaanalytics/Output v4-Finalized Orignial/ISS Pipelines/Output/merged", "ISS_Service.xlsx")

        # operations to be performed if the intervention input by the user is 6
        elif 'STAFF AVAILABILITY' in PHC_fold[i] and 'STAFF AVAILABILITY' in SHF_fold[i]:
            # rename all mapped SHF column names to be uniform with the corresponding PHC column names
            shf_file.rename({
              'grp1-human_resources-staff_availability-group32-followup_how_many_registered_midwives':'grp1-humanResources-staffAvailability-no_registered_midwives_nurses',
              'grp1-human_resources-staff_availability-group36-followup_how_many_laboratory_technicians':'grp1-humanResources-staffAvailability-no_labtech',
              'grp1-human_resources-staff_availability-group35-followup_how_many_pharmacy_technicians':'grp1-humanResources-staffAvailability-no_pharmtech',
              'grp1-human_resources-staff_availability-group39-followup_how_many_community_health_officers':'grp1-humanResources-staffAvailability-no_CHO',
              'grp1-human_resources-staff_availability-group31-followup_how_many_medical_officers':'grp1-humanResources-staffAvailability-no_medical_officers'
            }, axis =1, inplace = True)

            # concatenate / append both PHC and SHF files and output as a single CSV file
            # pd.concat([phc_file,shf_file],axis=0, ignore_index=True).to_csv(f'../output/merged/Complete_Staff_Availability.csv')

            staffav_df = pd.concat([phc_file,shf_file],axis=0, ignore_index=True)
            staffav_df2 = staffav_df.drop_duplicates(keep='first') #remove duplicates

            #reposition year and source
            yr = staffav_df2.pop('year')
            sc = staffav_df2.pop('source')
            id = staffav_df2.pop('id')
            staffav_df2=pd.concat([id,staffav_df2,yr,sc],axis=1)


            # Upload df as xlsx to Azure
            upload_excel(staffav_df2, "hefaanalytics/Output v4-Finalized Orignial/ISS Pipelines/Output/merged", "ISS_staff_availability.xlsx")

        # operations to be performed if the intervention input by the user is 6
        elif 'STAFF TRAINING' in PHC_fold[i] and 'STAFF TRAINING' in SHF_fold[i]:
            # rename all mapped SHF column names of the intervention to be uniform with the corresponding PHC column names
            shf_file.rename({
              'grp1-human_resources-staff_training-group57-staff_trained_on_malaria_management':'grp1-humanResources-staffTraining-num_trained_malaria',
              'grp1-human_resources-staff_training-group53-staff_trained_on_youth_friendly_health_services':'grp1-humanResources-staffTraining-num_trained_youthfriendly_centre',
              'grp1-human_resources-staff_training-group52-staff_trained_on_fp_clms_and_ctu_':'grp1-humanResources-staffTraining-num_trained_famplanning',
              'grp1-human_resources-staff_training-group50-staff_trained_on_hiv_aids_management':'grp1-humanResources-staffTraining-num_trained_HIV',
              'grp1-human_resources-staff_training-group49-staff_trained_on_integrated_management_of_childhood_illness':'grp1-humanResources-staffTraining-num_trained_imci',
              'grp1-human_resources-staff_training-group54-staff_trained_on_using_nhmis_version_19_tools':'grp1-humanResources-staffTraining-num_trained_hmis_tool'
            }, axis = 1, inplace = True)

            # concatenate / append both PHC and SHF files and output as a single CSV file
            # pd.concat([phc_file,shf_file],axis=0, ignore_index=True).to_csv(f'../output/merged/Complete_Staff_Trained.csv')

            staff_train_df = pd.concat([phc_file,shf_file],axis=0, ignore_index=True)
            staff_train_df2 = staff_train_df.drop_duplicates(keep='first') # remove duplicates

            #reposition year and source
            yr = staff_train_df2.pop('year')
            sc = staff_train_df2.pop('source')
            id = staff_train_df2.pop('id')
            staff_train_df2=pd.concat([id,staff_train_df2,yr,sc],axis=1)

            # Upload df as xlsx to Azure
            upload_excel(staff_train_df2, "hefaanalytics/Output v4-Finalized Orignial/ISS Pipelines/Output/merged", "staff_training_pipeline.xlsx")



# In[34]:


appending()


# In[35]:


# This function checks if files exist. If it exists, it appends. If it doesnt, it uploads the file.

def check_and_upload_file(df, outputPath, filename):
  blob_client = blob_service_client.get_blob_client(container=outputPath, blob=filename)

    #reposition id, year and source
  yr = df.pop('year')
  sc = df.pop('source')
  id = df.pop('id')
  df=pd.concat([id,df,yr,sc],axis=1)


  if not blob_client.exists():
    # If the file doesn't exist, create a new one
    dataframe_Buffer = io.BytesIO()
    df.to_excel(dataframe_Buffer, index=False)
    df_value = dataframe_Buffer.getvalue()
    blob_client.upload_blob(df_value, overwrite=True)
    print("File doesn't exist and has been uploaded")

  else:
    # If the file exists, append the data to the existing file
    stream = blob_client.download_blob().content_as_bytes()
    df2 = pd.read_excel(io.BytesIO(stream))
    df2 = df2.reset_index(drop=True)

    # df2 = df2.append(df)
    # df3 = df2.drop_duplicates(keep='first') # drop duplicates

    # Use pd.concat instead of df.append as df.append is deprecated
    df3 = pd.concat([df2, df]).drop_duplicates(keep='first')


    # Upload excel file
    dataframe_Buffer = io.BytesIO()
    df3.to_excel(dataframe_Buffer, index=False)
    df3_value = dataframe_Buffer.getvalue()
    blob_client.upload_blob(df3_value, overwrite=True)
    print("File exists and has been appended")


# In[42]:


def filter_each_intervention():
  import pandas as pd
  path = sorted_blob_list[0].name
#   path = "PHC or SHF"   #For Local Testing
  if 'PHC' in path:
    # List the blobs with .xlsx extension in the PHC folder
    PHC_fold = sorted([blob.name for blob in blob_service_client.get_container_client(container_name).list_blobs(name_starts_with='Output v4-Finalized Orignial/ISS Pipelines/Output/PHC/') if blob.name.endswith('.xlsx')])


    all_phc_num = ['grp1-humanResources-staffTraining-trained_HIV',
              'grp1-humanResources-staffTraining-trained_malaria',
              'grp1-humanResources-staffTraining-trained_Tuberculosis',
              'grp1-humanResources-staffTraining-trained_famplanning',
              'grp1-humanResources-staffTraining-trained_youthfriendly_centre',
              'grp1-humanResources-staffTraining-trained_hmis_tool',
              'grp1-humanResources-staffTraining-trained_intergrated_mnch',
              'grp1-humanResources-staffTraining-trained_ganc',
              'grp1-humanResources-staffTraining-trained_genderbased_violence',
              'grp1-humanResources-staffTraining-trained_inventory_management',
              'grp1-humanResources-staffAvailability-no_registered_midwives_nurses',
              'grp1-humanResources-staffAvailability-no_jchew',
              'grp1-humanResources-staffAvailability-no_labtech',
              'grp1-humanResources-staffAvailability-no_pharmtech',
              'grp1-humanResources-staffAvailability-no_chew',
              'grp1-humanResources-staffAvailability-no_CHO',
              'grp1-humanResources-staffAvailability-no_medical_officers',
              'grp1-humanResources-staffAvailability-num_Community_Volunteers_are_available'
              ]

    for filename in PHC_fold:
      phc_num = ['id','year', 'source']
      phc_binary = ['id','year', 'source']

      phc_blob_client = blob_service_client.get_blob_client(container_name, filename)
      stream = phc_blob_client.download_blob().content_as_bytes()
      df = pd.read_excel(io.BytesIO(stream))

      # Check if any column names match all_phc_num
      matching_columns = [col for col in df.columns if col in all_phc_num]
      non_matching_columns = [col for col in df.columns if col not in all_phc_num]

      phc_num = phc_num + matching_columns
      phc_num = list(set(phc_num))

      phc_binary = phc_binary + non_matching_columns
      phc_binary = list(set(phc_binary))

      if len(phc_num)>3:
        df_num = df[phc_num]
        df_num.reset_index(drop=True, inplace=True)
        df_num.to_excel('h_' + filename.split('/')[-1])
        check_and_upload_file (df_num , "hefaanalytics/Output v4-Finalized Orignial/ISS Pipelines/Output/Filtered_PHC", 'h_' + filename.split('/')[-1])

      if len(phc_binary)>3:
        df_bin = df[phc_binary]
        df_bin.reset_index(drop=True, inplace=True)
        check_and_upload_file (df_bin , "hefaanalytics/Output v4-Finalized Orignial/ISS Pipelines/Output/Filtered_PHC", 'd_' + filename.split('/')[-1])
        # df_bin.to_excel('d_' + filename.split('/')[-1])


# Filter for SHF Data

  if 'SHF' in path:
    # List the blobs with .xlsx extension in the SHF folder
    SHF_fold = sorted([blob.name for blob in blob_service_client.get_container_client(container_name).list_blobs(name_starts_with='Output v4-Finalized Orignial/ISS Pipelines/Output/SHF/') if blob.name.endswith('.xlsx')])


    all_shf_num = ['grp1-human_resources-staff_availability-group32-followup_how_many_registered_midwives',
              'grp1-human_resources-staff_availability-group33-followup_how_many_registered_general_nurses',
              'grp1-human_resources-staff_availability-group36-followup_how_many_laboratory_technicians',
              'grp1-human_resources-staff_availability-group35-followup_how_many_pharmacy_technicians',
              'grp1-human_resources-staff_availability-group39-followup_how_many_community_health_officers',
              'grp1-human_resources-staff_availability-group31-followup_how_many_medical_officers'
              ]

    for filename in SHF_fold:
      shf_num = ['id','year', 'source']
      shf_binary = ['id','year', 'source']

      shf_blob_client = blob_service_client.get_blob_client(container_name, filename)
      stream = shf_blob_client.download_blob().content_as_bytes()
      df = pd.read_excel(io.BytesIO(stream))

      # Check if any column names match all_shf_num
      matching_columns = [col for col in df.columns if col in all_shf_num]

      # Columns that dont match all_shf_num
      non_matching_columns = [col for col in df.columns if col not in all_shf_num]


      shf_num = shf_num + matching_columns
      shf_num = list(set(shf_num))  # make list unique

      shf_binary = shf_binary + non_matching_columns
      shf_binary = list(set(shf_binary)) # make list unique

      if len(shf_num)>3:
        df_num = df[shf_num]
        df_num.reset_index(drop=True, inplace=True)
        df_num.to_excel('h_' + filename.split('/')[-1])
        check_and_upload_file (df_num , "hefaanalytics/Output v4-Finalized Orignial/ISS Pipelines/Output/Filtered_SHF", 'h_' + filename.split('/')[-1])

      if len(shf_binary)>3:
        df_bin = df[shf_binary]
        df_bin.reset_index(drop=True, inplace=True)
        check_and_upload_file (df_bin , "hefaanalytics/Output v4-Finalized Orignial/ISS Pipelines/Output/Filtered_SHF", 'd_' + filename.split('/')[-1])



# In[43]:


filter_each_intervention()


# In[ ]:





# In[44]:


def merge_filtered_interventions():
  # List the blobs with .xlsx extension in the PHC folder
    PHC_fold = sorted([blob.name for blob in blob_service_client.get_container_client(container_name).list_blobs(name_starts_with='Output v4-Finalized Orignial/ISS Pipelines/Output/Filtered_PHC/') if blob.name.endswith('.xlsx')])
    # List the blobs with .xlsx extension in the SHF folder
    SHF_fold = sorted([blob.name for blob in blob_service_client.get_container_client(container_name).list_blobs(name_starts_with='Output v4-Finalized Orignial/ISS Pipelines/Output/Filtered_SHF/') if blob.name.endswith('.xlsx')])

    for i in range(len(PHC_fold)):

        # Download the PHC Excel file from Azure Blob Storage
        phc_blob_client = blob_service_client.get_blob_client(container_name, PHC_fold[i])
        stream = phc_blob_client.download_blob().content_as_bytes()
        phc_file = pd.read_excel(io.BytesIO(stream))

        # Download the SHF file from Azure Blob Storage
        shf_blob_client = blob_service_client.get_blob_client(container_name, SHF_fold[i])
        stream = shf_blob_client.download_blob().content_as_bytes()
        shf_file = pd.read_excel(io.BytesIO(stream))





        if "d_EQUIPMENT.xlsx" in PHC_fold[i].replace("PHC ", "").split("/")[-1] and "d_EQUIPMENT.xlsx" in SHF_fold[i].replace("SHF ", "").split("/")[-1]:
          shf_file.rename({'ggrp3-childbirth_labour-equipment_and_supplies_c-zzz55-delivery_packs':'chilhbirth_mm5-equipment_mm5-delivery_bed_mm5',
                    'ggrp1-adolescent_prepregnancy-equipment_and_supplies_a-group68-is_oxygen_available':'chilhbirth_mm5-equipment_mm5-Oxygen_mm5',
                    'ggrp3-childbirth_labour-equipment_and_supplies_c-group83-oxygen':'chilhbirth_mm5-equipment_mm5-Ambu_bag_mask_mm5',
                    'ggrp2-pregnancy-equipment_and_supplies_b-group75-sphygmomanometer':'pregnancy_mm4-equipment_mm4-Sphygmanometer_mm4',
                    'ggrp3-childbirth_labour-equipment_and_supplies_c-zzz55-sphygmomanometer':'chilhbirth_mm5-equipment_mm5-Sphygmomanometer_and_Stethoscope_mm5',
                    'ggrp3-childbirth_labour-equipment_and_supplies_c-group84-adult_stethoscopes_and_fetoscopes':'pregnancy_mm4-equipment_mm4-Fetal_and_Adult_Stethoscope_mm4',
                    'ggrp3-childbirth_labour-equipment_and_supplies_c-group85-clinical_thermometers':'chilhbirth_mm5-equipment_mm5-thermometer_mm5',
                    'ggrp5-newborn_care-equipment_and_supplies_e-zzz133-facility_have_thermometers':'newborn_care_mm7-equipment_mm7-Low_reading_thermomether_mm7',
                    'ggrp16-laboratoryServices-facility_safety-group260-bleach_and_other_disinfectants':'laboratoryServices-m102-bleachAndDisinfectants',
                    'ggrp2-pregnancy-equipment_and_supplies_b-zzz20-digital_blood_pressure_device_traffic_light_':'pregnancy_mm4-equipment_mm4-Digital_Blood_Pressure_Device_mm4',
                    'ggrp3-childbirth_labour-equipment_and_supplies_c-group85-weighing_scales':'childhealth_mm8-equipment_mm8-Infant_Weighing_Scales_mm8',
                    'ggrp2-pregnancy-equipment_and_supplies_b-zzz20-weighing_scale':'pregnancy_mm4-equipment_mm4-Weighing_Scale_mm4'}, axis=1, inplace = True)


          equipment_df = pd.concat([phc_file, shf_file], axis=0, ignore_index=True)
          equipment_df2 = equipment_df.drop_duplicates(keep='first') # remove duplicates
          yr = equipment_df2.pop('year')
          sc = equipment_df2.pop('source')
          id = equipment_df2.pop('id')
          equipment_df2=pd.concat([id,equipment_df2,yr,sc],axis=1)
          # Upload df as xlsx to Azure
          upload_excel(equipment_df2, "hefaanalytics/Output v4-Finalized Orignial/ISS Pipelines/Output/Filtered_merged", "d_ISS_Equipment.xlsx")

        # ............


        elif "d_INFRASTRUCTURE.xlsx" in PHC_fold[i].replace("PHC ", "").split("/")[-1] and "d_INFRASTRUCTURE.xlsx" in SHF_fold[i].replace("SHF ", "").split("/")[-1]:
          shf_file.rename({'grp1-external_env_and_infrastructure:group8-functional_source_of_power_supply_to_the_facility':'grp1-ExternalEnvironmentPhysicalInfrastructure-functional_source_of_power_supply_to_this_facility',
            'grp1-external_env_and_infrastructure-group8-facility_fenced':'grp1-ExternalEnvironmentPhysicalInfrastructure-health_facility_fenced',

            'grp1-external_env_and_infrastructure-ff17-accessible_road_to_the_facility':'grp1-ExternalEnvironmentPhysicalInfrastructure-Accessible_road_to_the_facility',
            'grp1-external_env_and_infrastructure-group6-facilitys_sewage_disposal_system_adequate':'grp1-ExternalEnvironmentPhysicalInfrastructure-sewage_disposal_system_adequate',
            'grp1-external_env_and_infrastructure-group5-facility_have_potable_water':'grp1-ExternalEnvironmentPhysicalInfrastructure-potable_water_available',
            'grp1-external_env_and_infrastructure-group6-facility_have_a_sharp_disposal_system_in_place':'grp1-ExternalEnvironmentPhysicalInfrastructure-sharp_disposal_system',
            'grp1-external_env_and_infrastructure-group5-facility_well_ventilated':'grp1-ExternalEnvironmentPhysicalInfrastructure-facility_well_ventilated',
            'grp1-external_env_and_infrastructure-group5-roof_intact_with_no_leaking_part':'grp1-ExternalEnvironmentPhysicalInfrastructure-roof_intact_with_no_leaking',
            'grp1-external_env_and_infrastructure-ff17-walls_free_of_cracks_and_crevices':'grp1-ExternalEnvironmentPhysicalInfrastructure-walls_free_of_cracks',
            'grp1-external_env_and_infrastructure-group7-facility_carry_out_quarterly_pest_control_activities':'grp1-ExternalEnvironmentPhysicalInfrastructure-quaterly_rodent_and_pest_control',
            'grp1-external_env_and_infrastructure-group8-functional_source_of_power_supply_to_the_facility':'grp1-ExternalEnvironmentPhysicalInfrastructure-functional_source_of_power_supply_to_this_facility',
            'grp1-external_env_and_infrastructure-group6-facilitys_waste_disposal_system_adequate':'grp1-ExternalEnvironmentPhysicalInfrastructure-waste_disposal_system_adequate',
            'grp1-external_env_and_infrastructure-ff17-facility_environment_generally_neat_and_tidy': 'grp1-ExternalEnvironmentPhysicalInfrastructure-environment_generally_tidy'
            }, axis=1, inplace = True)

            # concatenate / append both PHC and SHF files and output as a single CSV file
            # pd.concat([phc_file,shf_file], axis=0, ignore_index=True).to_csv(f'../output/merged/Complete_Infrastructure.csv')

          infra_df = pd.concat([phc_file,shf_file], axis=0, ignore_index=True)
          infra_df2 = infra_df.drop_duplicates(keep='first') # remove duplicates


          #reposition year and source
          yr = infra_df2.pop('year')
          sc = infra_df2.pop('source')
          id = infra_df2.pop('id')
          infra_df2=pd.concat([id,infra_df2,yr,sc],axis=1)

          # Upload df as xlsx to Azure
          upload_excel(infra_df2, "hefaanalytics/Output v4-Finalized Orignial/ISS Pipelines/Output/Filtered_merged", "d_infrastructure_pipeline.xlsx")


        elif "d_LOGISTICS.xlsx" in PHC_fold[i].replace("PHC ", "").split("/")[-1] and "d_LOGISTICS.xlsx" in SHF_fold[i].replace("SHF ", "").split("/")[-1]:
          shf_file.rename({
            'ggrp18-logistics_management-data_verification-medicine_supplies_equipment_l-group345-albendazole':'logisticsManagement-albendazole',
            'ggrp11-nutrition-medicines_and_other_health_technologies_i-group239-albendezole_mebendazole':'logisticsManagement-mebendazole',
            'ggrp18-logistics_management-data_verification-medicine_supplies_equipment_l-group344-act':'logisticsManagement-acts',
            'ggrp18-logistics_management-data_verification-medicine_supplies_equipment_l-group345-dispersible_amoxicillin_tablets':'logisticsManagement-dispersible',
            'ggrp18-logistics_management-data_verification-medicine_supplies_equipment_l-group348-oral_pills':'logisticsManagement-contraceptive',
            'ggrp18-logistics_management-data_verification-medicine_supplies_equipment_l-group339-iron_and_folic_acid_tablet':'logisticsManagement-folic',
            'ggrp1-adolescent_prepregnancy-equipment_and_supplies_a-ff230-are_male_condoms_available':'logisticsManagement-condomMale',
            'ggrp1-adolescent_prepregnancy-equipment_and_supplies_a-ff230-are_female_condoms_available':'logisticsManagement-condomFemale',
            'ggrp6-childhealth-medicines_and_other_health_technologies_f-group120-facility_have_vitamin_a':'logisticsManagement-vitaminA',
            'ggrp18-logistics_management-data_verification-medicine_supplies_equipment_l-group341-tetanus_toxoid':'logisticsManagement-tetanus',
            'ggrp18-logistics_management-data_verification-medicine_supplies_equipment_l-group346-zinc_tablet':'logisticsManagement-zincORS',
            'ggrp18-logistics_management-data_verification-medicine_supplies_equipment_l-group346-oxytocine':'logisticsManagement-coldChainItems'},axis=1, inplace=True)


          # concatenate / append both PHC and SHF files and output as a single CSV file
          # pd.concat([phc_file,shf_file],axis=0, ignore_index=True).to_csv(f'../output/merged/Complete_Logistics.csv')

          logistics_df = pd.concat([phc_file,shf_file],axis=0, ignore_index=True)
          logistics_df2 = logistics_df.drop_duplicates(keep='first') # remove duplicates


            #reposition year and source
          yr = logistics_df2.pop('year')
          sc = logistics_df2.pop('source')
          id = logistics_df2.pop('id')
          logistics_df2=pd.concat([id,logistics_df2,yr,sc],axis=1)

          # Upload df as xlsx to Azure
          upload_excel(logistics_df2, "hefaanalytics/Output v4-Finalized Orignial/ISS Pipelines/Output/Filtered_merged", "d_Logistics_management_pipeline.xlsx")


        elif "d_SERVICES.xlsx" in PHC_fold[i].replace("PHC ", "").split("/")[-1] and "d_SERVICES.xlsx" in SHF_fold[i].replace("SHF ", "").split("/")[-1]:
          shf_file.rename({
            'ggrp3-childbirth_labour-service_provision_c-zzz37-provide_care_during_labour_and_delivery':'chilhbirth_mm5-serviceprovision_mm5-Care_during_labour_and_childbirth_mm5',
            'ggrp2-pregnancy-service_provision_b-zzz2-nutritional_counselling_and_food_demonstration':'nutrition_mm13-patient_management_mm13-nutrition_counselling_mm13',
            'ggrp10-immunization-service_provision_h-group214-followup_planned_routine_immunization_session_being_conducted':'immunization_mm12-rew_micrplan_in_use_mm12'
            }, axis =1, inplace = True)

            # concatenate / append both PHC and SHF files and output as a single CSV file
            # pd.concat([phc_file,shf_file],axis=0, ignore_index=True).to_csv(f'../output/merged/Complete_Services.csv')

          services_df = pd.concat([phc_file,shf_file],axis=0, ignore_index=True)
          services_df2 = services_df.drop_duplicates(keep='first') #remove duplicates


          #reposition year and source
          yr = services_df2.pop('year')
          sc = services_df2.pop('source')
          id = services_df2.pop('id')
          services_df2=pd.concat([id,services_df2,yr,sc],axis=1)


                #rename columns with truncated names
          services_df3=services_df2.rename({"ggrp8-hiv_preventn_treatmnt-adolescent_friendly_and_youth_wellness_services-group153-do_clients_receive_sexually_transmitted_infection_screening":"grp8-hiv_prvntn_trtmnt-adlscnt_frndly_&_youth_welnes_servics-grp153-do_clients_recive_sxualy_trnsmitd_infctn_screning",

          "ggrp8-hiv_preventn_treatmnt-pediatric_art-part-are_all_confirmed_hiv_infected_children_with_treatment_failure_switched_to_2nd_line_art":"grp8-hiv_prvntn_treatmnt-pdiatrc_art-prt-are_al_cnfrmed_hiv_infctd_chldrn_wit_tretmnt_failre_swtchd_to_2nd_line_art",

          "ggrp8-hiv_preventn_treatmnt-prevention_of_mother_to_child_transmission-ptt-do_all_pregnant_women_receive_counselling_and_testing_during_antenatal_care":"grp8-hiv_prevntn_treatmnt-prevntn_of_mthr_to_chld_trnsmssn-ptt-do_al_prgnnt_womn_recive_cnseling_&_tstng_durng_antnatl_cre",

          "ggrp3-childbirth_labour-service_provision_c-zzz37-manage_obstetric_complications_in_childbirth_and_immediate_post_partum_period":"grp3-chldbrth_labr-srvce_prvisn_c-z37-mnage_obstetrc_cmplicatns_in_chldbrth_&_imediate_pst_partm_perod",

          "ggrp8-hiv_preventn_treatmnt-prevention_of_mother_to_child_transmission-group165-are_all_heis_provided_with_arv_prophylaxis_at_birth":"grp8-hiv_prvntn_treatmnt-prvntn_of_mothr_to_chld_trnsmisn-grp165-are_al_his_prvidd_wth_arv_prphylxis_at_brth",

          "ggrp1-adolescent_prepregnancy-service_provision_a-group62-carry_out_prevention_and_management_of_hiv_sexually_transmitted_infection":"grp1-adlscnt_prepregnancy-servce_prvisn_a-grp62-cary_out_prevntn_&_managmnt_of_hiv_sxualy_trnsmitd_infctn",

          "ggrp1-adolescent_prepregnancy-service_provision_a-group62-carry_out_appropriate_management_of_infertile_couple_including_hiv_discordant_couples":"grp1-adolscnt_preprgnncy-service_prvisn_a-grp62-cary_out_apropriate_mnagmnt_of_infrtile_couple_incldng_hiv_dscrdnt_coupls",

          "ggrp8-hiv_preventn_treatmnt-pediatric_art-part-are_all_confirmed_hiv_infected_children_with_treatment_failure_switched_to_2nd_line_art.1":"grp8-hiv_prevntn_treatmnt-pediatrc_art-prt-are_all_cnfirmd_hiv_infctd_childrn_with_treatmnt_failre_swtchd_to_2nd_lne_art1",

          "ggrp8-hiv_preventn_treatmnt-prevention_of_mother_to_child_transmission-ptt-do_all_pregnant_women_receive_counselling_and_testing_during_antenatal_care.1":"grp8-hiv_prevntn_trtmnt-prventn_of_mothr_to_child_trnsmisn-pt-do_al_prgnnt_women_recive_cnseling_&_tstng_drng_antnatl_cre1"}, axis=1)


          # Upload df as xlsx to Azure
          upload_excel(services_df3, "hefaanalytics/Output v4-Finalized Orignial/ISS Pipelines/Output/Filtered_merged", "d_ISS_Service.xlsx")



        elif "d_STAFF TRAINING.xlsx" in PHC_fold[i].replace("PHC ", "").split("/")[-1] and "d_STAFF TRAINING.xlsx" in SHF_fold[i].replace("SHF ", "").split("/")[-1]:
          shf_file.rename({
              'grp1-human_resources-staff_training-group57-staff_trained_on_malaria_management':'grp1-humanResources-staffTraining-num_trained_malaria',
              'grp1-human_resources-staff_training-group53-staff_trained_on_youth_friendly_health_services':'grp1-humanResources-staffTraining-num_trained_youthfriendly_centre',
              'grp1-human_resources-staff_training-group52-staff_trained_on_fp_clms_and_ctu_':'grp1-humanResources-staffTraining-num_trained_famplanning',
              'grp1-human_resources-staff_training-group50-staff_trained_on_hiv_aids_management':'grp1-humanResources-staffTraining-num_trained_HIV',
              'grp1-human_resources-staff_training-group49-staff_trained_on_integrated_management_of_childhood_illness':'grp1-humanResources-staffTraining-num_trained_imci',
              'grp1-human_resources-staff_training-group54-staff_trained_on_using_nhmis_version_19_tools':'grp1-humanResources-staffTraining-num_trained_hmis_tool'
            }, axis = 1, inplace = True)

          # concatenate / append both PHC and SHF files and output as a single CSV file
          # pd.concat([phc_file,shf_file],axis=0, ignore_index=True).to_csv(f'../output/merged/Complete_Staff_Trained.csv')

          staff_train_df = pd.concat([phc_file,shf_file],axis=0, ignore_index=True)
          staff_train_df2 = staff_train_df.drop_duplicates(keep='first') # remove duplicates

          #reposition year and source
          yr = staff_train_df2.pop('year')
          sc = staff_train_df2.pop('source')
          id = staff_train_df2.pop('id')
          staff_train_df2=pd.concat([id,staff_train_df2,yr,sc],axis=1)

          # Upload df as xlsx to Azure
          upload_excel(staff_train_df2, "hefaanalytics/Output v4-Finalized Orignial/ISS Pipelines/Output/Filtered_merged", "d_staff_training_pipeline.xlsx")


        elif "h_STAFF AVAILABILITY.xlsx" in PHC_fold[i].replace("PHC ", "").split("/")[-1] and "h_STAFF AVAILABILITY.xlsx" in SHF_fold[i].replace("SHF ", "").split("/")[-1]:
          shf_file.rename({
              'grp1-human_resources-staff_availability-group32-followup_how_many_registered_midwives':'grp1-humanResources-staffAvailability-no_registered_midwives_nurses',
              'grp1-human_resources-staff_availability-group36-followup_how_many_laboratory_technicians':'grp1-humanResources-staffAvailability-no_labtech',
              'grp1-human_resources-staff_availability-group35-followup_how_many_pharmacy_technicians':'grp1-humanResources-staffAvailability-no_pharmtech',
              'grp1-human_resources-staff_availability-group39-followup_how_many_community_health_officers':'grp1-humanResources-staffAvailability-no_CHO',
              'grp1-human_resources-staff_availability-group31-followup_how_many_medical_officers':'grp1-humanResources-staffAvailability-no_medical_officers'
            }, axis =1, inplace = True)

            # concatenate / append both PHC and SHF files and output as a single CSV file
            # pd.concat([phc_file,shf_file],axis=0, ignore_index=True).to_csv(f'../output/merged/Complete_Staff_Availability.csv')

          staffav_df = pd.concat([phc_file,shf_file],axis=0, ignore_index=True)
          staffav_df2 = staffav_df.drop_duplicates(keep='first') #remove duplicates

          #reposition year and source
          yr = staffav_df2.pop('year')
          sc = staffav_df2.pop('source')
          id = staffav_df2.pop('id')
          staffav_df2=pd.concat([id,staffav_df2,yr,sc],axis=1)


          # Upload df as xlsx to Azure
          upload_excel(staffav_df2, "hefaanalytics/Output v4-Finalized Orignial/ISS Pipelines/Output/Filtered_merged", "h_ISS_staff_availability.xlsx")






# In[45]:


merge_filtered_interventions()


# In[ ]:






Transformation and upload complete.
File exists and has been appended
File exists and has been appended
File exists and has been appended
File exists and has been appended
File exists and has been appended
File exists and has been appended
Output v4-Finalized Orignial/ISS Pipelines/Input/PHC_APR_2024.csv
File exists and has been appended
File exists and has been appended
File exists and has been appended
File exists and has been appended
File exists and has been appended
File exists and has been appended
